In [1]:
import os
import sys
import subprocess
from pathlib import Path

In [2]:
#element := (folder,entry,grading,testcases)
#path example ./correct_or_not/problem_name/year/submission
global log 

benchmark_info = [
    ("checkMetro","checkMetro", "", "testcases"),
    #("KoreaUniv_diff","grading","grading.ml","testcases"),
]

test_folders = ["checkMetro"]

log = {}
delimit = b"\n-----------------------------\nRepair result\n-----------------------------\n"

In [3]:
def gen_command(submission,sols,entry,grading,specs):
    opt_submission = " -submission " + submission
    opt_solutions = " -solutions " + sols
    opt_entry = " -entry " + entry
    opt_testcases = " -testcases " + specs
    opt_grading = ""
    if grading != "":
        opt_grading = " -grading " + grading
    
    pgm = "engine/main.native -experiment "
    command = pgm + opt_submission + opt_solutions + opt_entry + opt_testcases + opt_grading
    #print(command)
    return command

In [4]:
def repair_result_parse(data):
    ''' input format(string)
    
    "benchmarks_correct/KoreaUniv_diff/18/sub9.ml
    Size of repair Cand : 1
    Fail to Repair"
    
    OR
    
    "benchmarks_correct/KoreaUniv_diff/18/sub9.ml
    Size of repair Cand : 1
    <code>..."
    '''
    parsed = data.split(delimit)
    if len(parsed)==1:
        parsed = [False,parsed[0].strip().rsplit(b'\n', maxsplit=1)]
    else :
        parsed = list(map(bytes.strip,parsed)))
    return parsed

In [5]:
def execute(submission,sols,entry,grading,specs):
    global log
    command = gen_command(submission,sols,entry,grading,specs)
    p = subprocess.Popen(command,stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=True)
    out, err = p.communicate()
    log[submission] = {}
    if not err:
        result = out.split(b"@#$@#$@#$")
        # result[1]: "Matched Solution : 5\n"
        log[submission]['num_of_matched_sol'] = int(result[1].split(b"Matched Solution :")[1])
        if log[submission]['num_of_matched_sol'] != 0 :
            repair_results = result[2:]
            # \n'path'\n'repairCand'\n'fail'or'code'\n
            log[submission]['repair_results'] = []
            parsed_results = map(repair_result_parse, repair_results)
            for is_patched,meta,code in parsed_results:
                path, cand = meta.splitlines()
                if is_patched:
                    log[submission]['repair_results'].append((is_patched,path,code))
                else :
                    log[submission]['repair_results'].append((is_patched,path,""))
    else:
        log[submission]['error_message'] = err
    return command
   

In [6]:
def print_log(log):
    print(log)

In [7]:
def main() :
    #suppose this script excuted in LearnML root directory 'your_path/LearnML'
    global log
    root = Path('.')
    correct = root / "benchmarks_correct"
    incorrect = root / "benchmarks_incorrect"
    
    for (prob,entry,grading,specs) in benchmark_info:
        log = {}
        correct_prob = correct / prob
        incorrect_prob = incorrect / prob
        testcases = str(correct / prob / specs)
        if grading != "":
            grading_path = str(correct / prob / grading)
        else :
            grading_path = ""
        sols = str(correct_prob)
        for year in incorrect_prob.iterdir():
            if year.is_dir():
                for submission in year.iterdir():
                    sub = str(submission)
                    a = execute(sub,sols,entry,grading_path,testcases)
                    
        print_log(log)
                                        

In [8]:
main()

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
log

In [ ]:
log['benchmarks_incorrect/checkMetro/13/sub67.ml']['test']

In [ ]:
'''
if __name__ == '__main__':
    if len(sys.argv) != 2:
        print("wrong argument")
    else:
        main()
'''